### Automating Model Runs - Streamflow Capture Analysis

All groundwater pumped is balanced by removal of water somewhere, initially from storage in the aquifer and later from capture in the form of increase in recharge and decrease in discharge (Leake and others, 2010). Capture that results in a loss of water in streams, rivers, and wetlands now is a concern in many parts of the United States. Hydrologists commonly use analytical and numerical approaches to study temporal variations in sources of water to wells for select points of interest. Much can be learned about coupled surface/groundwater systems, however, by looking at the spatial distribution of theoretical capture for select times of interest. Development of maps of capture requires (1) a reasonably well-constructed transient or steady state model of an aquifer with head-dependent flow boundaries representing surface water features or evapotranspiration and (2) an automated procedure to run the model repeatedly and extract results, each time with a well in a different location. In this exercise, we will perform a streamflow capture analysis of the Freyberg model domain by developing a MODFLOW model, running it as many times as there are active model cells, and then creating a streamflow capture fraction map to summarize the results.

[Leake, S. A., Reeves, H. W. and Dickinson, J. E. (2010), A New Capture Fraction Method to Map How Pumpage Affects Surface Water Flow. Ground Water, 48: 690–700. doi: 10.1111/j.1745-6584.2010.00701.x](http://onlinelibrary.wiley.com/doi/10.1111/j.1745-6584.2010.00701.x/abstract)



In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import flopy

#### Load existing freyberg model

The MODFLOW 6 version of the freyberg model is located in:

```
../../data/freyberg
```

The model name is `freyberg6`.

You should define the model workspace (`ws`) where the model files are, the model name (`name`), and the name and path of the model executable (`exe_name`). 

In [ ]:
os.getcwd()

### Set the location of the model data sets and the executables

_These paths are relative to the directory containing this notebook_  
`iah2021-brazil-mf6/notebooks/streamflow_depletion`

In [ ]:
ws = "../../data/freyberg/"
name = "freyberg6"
exe_name = "mf6"

# load simulation
sim = flopy.mf6.MFSimulation.load(sim_name=name, exe_name=exe_name, sim_ws=ws)

#### Change the model workspace and run the model

The model workspace can be changed using `sim.set_sim_path(ws)`, where `ws` is set to be `work/ex06`. Next write the simulation using `sim.write_simulation()` and run the model using `sim.run_simulation()`.

In [ ]:
ws = "temp/automation"
sim.set_sim_path(ws)

In [ ]:
sim.write_simulation()

In [ ]:
sim.run_simulation()

#### Extract the river results for the base model

Load the `SFR` observations from the groundwater flow model using the `gwf.sfr.output.obs().data` method.

First get the gwf model object so that we can add a new well package to perturb the stream flow in each cell. You can get a list of the available models in the simulation using `sim.model_names`. Get the gwf model object using `sim.get_model()`. The package names can be determined using `gwf.package_names`.

In [ ]:
gwf = sim.get_model("freyberg6")
gwf.package_names

In [ ]:
sfr = gwf.get_package("sfr_1")
obs = sfr.output.obs().data

The observation file contains a headwater and tailwater observations. The data returned by `gwf.sfr.output.obs().data` is a numpy recarray.

In [ ]:
headwater_base, tailwater_base = float(obs["HEADWATER"]), float(obs["TAILWATER"])
headwater_base, tailwater_base

#### Add additional wells and perform streamflow capture analysis

We will need the idomain so that we only add wells in active cells. The idomain can be retrieved using `gwf.dis.idomain.array`. It will be useful to have the number of rows and columns in the model.

In [ ]:
idomain = gwf.dis.idomain.array
idomain.shape

In [ ]:
nlay, nrow, ncol = idomain.shape
nlay, ncol, nrow

Make an array to store the values

In [ ]:
capture_shape = (2,) + idomain.shape
capture = np.zeros(capture_shape, dtype=np.float)

In [ ]:
capture_shape

#### Streamflow capture analysis code block
The code block below loops through every cell in the model and for each active cell adds a well in the current cell, rewrites the well file, reruns the model, extracts river leakage results from the model, and calculates the streamflow capture fraction for the cell. The model is run  with `silent=True` to suppress model output to the screen.

Streamflow capture is defined as 

$c_{k,i,j} = \frac{q_{k,i,j} - q_{{k,i,j}_{\text{base}}}}{|q_{\text{well}}|}$,

where $q_{\text{well}}$ is the pumping rate applied in each cell (use `-0.001`), $q_{k,i,j}$ is the net simulated river flux, and $q_{{k,i,j}_{\text{base}}}$ is the net simulated river flux from the base model. 

In [ ]:
wnam = gwf.name + '_cf.wel'
pname = 'cfwell'
qwell = -10.
failures = []
for k in range(nlay):
    for i in range(nrow):
        for j in range(ncol):
            # skip inactive cells
            if idomain[k, i, j] < 1:
                continue
            print(f'running k:{k}, i:{i}, j:{j}\r', end='')
            # make a new well package
            wel_spd = [[k, i, j, qwell]]
            wel2 = flopy.mf6.ModflowGwfwel(gwf, stress_period_data=wel_spd, pname=pname, filename=wnam)

            # write the simulation files
            sim.write_simulation(silent=True)

            # run the simulation
            success = sim.run_simulation(silent=True)
            if not success:
                failures.append((k, i, j))

            # process the results
            obs = sfr.output.obs().data
            headwater, tailwater = float(obs["HEADWATER"]), float(obs["TAILWATER"])
            cf_headwater = (headwater - headwater_base) / abs(qwell)
            cf_tailwater = (tailwater - tailwater_base) / abs(qwell)

            # add the value to the capture array
            capture[0, k, i, j] = cf_headwater
            capture[1, k, i, j] = cf_tailwater

            # remove the new well package so it can be readded
            gwf.remove_package(pname)

if len(failures) > 0:
    for k, i, j in failures:
        print('model failed at cell ({}, {}, {})'.format(k, i, j))

# Finally, plot the capture

In [ ]:
vmin, vmax = 0., 1.
fig, axes = plt.subplots(nrows=2, ncols=nlay, figsize=(8,8), constrained_layout=True)
iax = 0
for i in range(2):
    if i == 0:
        prefix = "Headwater"
    else:
        prefix = "Tailwater"
    for k in range(nlay):
        ax = axes.flatten()[iax]
        ax.set_aspect('equal')
        title = "{} - Layer {}".format(prefix, k+1)
        mm = flopy.plot.PlotMapView(model=gwf, ax=ax, layer=k)
        v = mm.plot_array(capture[i, k, :, :], masked_values=[0], cmap='jet_r', vmin=vmin, vmax=vmax)
        mm.plot_grid(lw=0.5, color='black')
        mm.plot_bc("SFR", alpha=0.5)
        mm.plot_bc("WEL", color="#FF33D7")
        mm.plot_bc("GHB", color="green")
        mm.plot_ibound()
        ax.set_title(title)
        if iax == 4:
            cbar = plt.colorbar(v, ax=ax, orientation="horizontal", shrink=1.)
            cbar.set_label("Capture Fraction")
        iax += 1
        